
### **Description**  

#### **Images Dataset**  
- **Author**: Pavan Sanagapati  
- **Source**: [Images Dataset on Kaggle](https://www.kaggle.com/datasets/pavansanagapati/images-dataset/data)  
- **Content**: This dataset contains images from various categories, including:  
  - Bikes  
  - Cars  
  - Cats  
  - Dogs  
  - Flowers  
  - Horses  
  - Humans  
- **Purpose**: The dataset is intended for tasks related to identifying *unknown* labels, making it suitable for applications in multi-class classification and open-set recognition problems.  

#### **X-ray Dataset(Pneumonia-Normal)**  
- **Author**: Paul Mooney  
- **Source**: [Chest X-ray Pneumonia Dataset on Kaggle](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/data)  
- **Content**: This dataset includes X-ray images categorized into two classes:  
  - Pneumonia  
  - Normal  
- **Purpose**: This dataset has been utilized for solving the problem of detecting unknown classes in medical imaging, particularly focusing on distinguishing between pneumonia and normal cases.


#### **X-ray Dataset(Tuberculosis-Normal)**
- **Author**: Tapendu Karmakar  
- **Source**: [Chest X-ray Dataset for Tuberculosis Segmentation](https://www.kaggle.com/datasets/iamtapendu/chest-x-ray-lungs-segmentation)  
- **Content**: This dataset includes X-ray images categorized into two classes:  
  - Tuberculosis  
  - Normal  
- **Purpose**: This dataset has been utilized for solving the problem of detecting unknown classes in medical imaging, particularly focusing on distinguishing between Tuberculosis and normal cases.

#### **X-ray Dataset(Tuberculosis)**
- **Author**: Tapendu Karmakar  
- **Source**: [Tuberculosis (TB) Chest X-ray Database](https://www.kaggle.com/datasets/tawsifurrahman/tuberculosis-tb-chest-xray-dataset)  
- **Content**: This dataset includes X-ray images categorized into two classes:  
  - Tuberculosis  
  - Normal  
- **Conflict**  : Normal class is taken from [Chest X-ray Pneumonia Dataset on Kaggle](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/data)
- **Purpose**: This dataset has been utilized for Increase of data volume in Tuberculosis class 


include and update the info I have Taken tuberculosis data from :

""" 
An official website of the United States government

Here's how you know

                                  PMC home page
                              
Search

Log in
Primary site navigation
Search PMC Full-Text Archive
Search PMC Full-Text Archive

Search in PMC
Advanced Search 
Journal List 
User Guide
As a library, NLM provides access to scientific literature. Inclusion in an NLM database does not imply endorsement of, or agreement with, the contents by NLM or the National Institutes of Health.
Learn more: PMC Disclaimer | PMC Copyright Notice
Quantitative Imaging in Medicine and Surgery logo
Quant Imaging Med Surg. 2014 Dec;4(6):475–477. doi: 10.3978/j.issn.2223-4292.2014.11.20
Two public chest X-ray datasets for computer-aided screening of pulmonary diseases
Stefan Jaeger 1,✉, Sema Candemir 1, Sameer Antani 1, Yì-Xiáng J Wáng 2, Pu-Xuan Lu 3, George Thoma 1
Author information
Article notes
Copyright and License information
PMCID: PMC4256233  PMID: 25525580 """


and 
insights from """"Received September 27, 2020, accepted October 12, 2020, date of publication October 15, 2020, date of current version October 30, 2020.
Digital Object Identifier 10.1109/ACCESS.2020.3031384
Reliable Tuberculosis Detection Using Chest
X-Ray With Deep Learning, Segmentation
and Visualization
TAWSIFUR RAHMAN 1
, AMITH KHANDAKAR 2
, (Senior Member, IEEE),
MUHAMMAD ABDUL KADIR 1
, KHANDAKER REJAUL ISLAM 3
,
KHANDAKAR F. ISLAM2
, RASHID MAZHAR 4,5, TAHIR HAMID5,6
,
MOHAMMAD TARIQUL ISLAM 7
, (Senior Member, IEEE), SAAD KASHEM8
,
ZAID BIN MAHBUB9
, MOHAMED ARSELENE AYARI10, AND
MUHAMMAD E. H. CHOWDHURY 2
, (Senior Member, IEEE)
1Department of Biomedical Physics and Technology, University of Dhaka, Dhaka 1000, Bangladesh
2Department of Electrical Engineering, Qatar University, Doha 2713, Qatar
3Department of Orthodontics, Bangabandhu Sheikh Mujib Medical University, Dhaka 1000, Bangladesh
4Thoracic Surgery, Hamad General Hospital, Doha 3050, Qatar
5Department of Medicine, Weill Cornell Medicine-Qatar, Doha 24811, Qatar
6Cardiology, Hamad General Hospital, Doha 3050, Qatar
7Department of Electrical, Electronic, and Systems Engineering, Universiti Kebangsaan Malaysia, Bangi 43600, Malaysia
8Faculty of Robotics and Advanced Computing, Qatar Armed Forces-Academic Bridge Program, Qatar Foundation, Doha 24404, Qatar
9Department of Mathematics and Physics, North South University, Dhaka 1229, Bangladesh
10College of Engineering, Qatar University, Doha 2713, Qatar
Corresponding author: Mohamed Arselene Ayari (arslana@qu.edu.qa)
This work was made possible by NPRP12S-0227-190164 from the Qatar National Research Fund, a member of Qatar Foundation, Doha,
Qatar. Open Access funding provided by the Qatar National Library :A. DATASETS DESCRIPTION
1) LUNG SEGMENTATION
In this work, Kaggle Chest X-ray images and corresponding
lung mask dataset [69] were used for training the lung segmentation models, where 704 X-ray images and their corresponding ground truth lung masks are available. All masks
VOLUME 8, 2020 191589
T. Rahman et al.: Reliable TB Detection Using CXR With Deep Learning, Segmentation, and Visualizatio
FIGURE 1. Architecture of A) original U-Net and B) modified U-Net.
FIGURE 2. Score-CAM heat map on a chest X-ray image showing that
different regions of the image were used in decision making by the CNN.
were annotated by expert radiologists; sample X-ray images
and masks are shown in Figure 4. There are 360 normal
X-ray images and 344 abnormal (infected lung) X-ray images
available in the dataset. Therefore, U-Net networks were
trained with both normal and abnormal images.
2) TB CLASSIFICATION
Four publicly accessible databases were used for TB classification problem. These are NLM dataset, Belarus dataset,
NIAID TB dataset and RSNA dataset:
NLM dataset: National Library of Medicine (NLM)
in U.S. [25] has made two lung X-ray datasets publicly
available: the Montgomery and Shenzhen datasets. The
Montgomery County (MC) and the Shenzhen, China (CHN)
databases are comprised of 138 and 667 posterior-anterior
(PA) chest X-ray images respectively. The resolution of
the images of MC database was either 4, 020 × 4.892
or 4, 892 × 4.020 pixels whereas that for CHN database
was variable but around 3000 × 3000 pixels. In the MC
database, out of 138 chest X-ray images, 58 images were
taken from different TB patients and 80 images were from
normal subjects. In the CHN database, out of 662 chest X-ray
images, 336 images were taken from different TB patients
191590 VOLUME 8, 2020
T. Rahman et al.: Reliable TB Detection Using CXR With Deep Learning, Segmentation, and Visualizatio
FIGURE 3. Overview of the complete system.
FIGURE 4. Example of X-ray images and corresponding ground truth lung
masks from Kaggle dataset.
and 324 images were from normal subjects. Therefore, in this
NLM database, there are 406 normal and 394 TB infected
X-ray images.
Belarus dataset: Belarus Set [70] was collected for a
drug resistance study initiated by the National Institute of
Allergy and Infectious Diseases, Ministry of Health, Republic of Belarus. The dataset contains 306 CXR images of
169 patients. Chest radiographs were taken using the Kodak
Point-of-Care 260 system and the resolution of the images
was 2248 × 2248 pixels. All the images of this database are
TB infected images.
NIAID TB dataset: NIAID TB portal program dataset [71],
which contains about 3000 TB positive CXR images from
about 3087 cases. All images were collected from seven
different countries and all images are in Portable Network
Graphics (PNG) format. In this study, we have used 2800 TB
positive CXR images out of 3000 images. 200 poor quality
images were discarded from this database.
RSNA CXR dataset: RSNA pneumonia detection challenge dataset [72], which is comprised of about 30,000 chest
X-ray images, where 10,000 images are normal and others are
abnormal and lung opacity images. All images are in Digital
Imaging and Communications in Medicine (DICOM) format.
To create a normal database of 3,500 chest X-ray images for
this study, 3,094 normal images were taken from this database
and rest of the 406 normal images were taken from the
NLM database. However, the number of TB infected images
by combining NLM and Belarus dataset was 700 and from
NIAID TB dataset was 2800. In total, there were 3500 TB
infected and 3500 normal X-ray images were used in th"





as well as 
"""
Tapendu Karmakar · Updated a month ago

New Notebook

Download

Chest X-ray Dataset for Tuberculosis Segmentation
Chest X-ray Organized Lung Segmentation Masks

source "https://www.kaggle.com/datasets/iamtapendu/chest-x-ray-lungs-segmentation" """




Category: NORMAL, Total: 5180, Train: 4870, Val: 51, Test: 259
Category: PNEUMONIA, Total: 3883, Train: 3651, Val: 38, Test: 194
Category: UNKNOWN, Total: 1357, Train: 1277, Val: 13, Test: 67
Category: TUBERCULOSIS, Total: 968, Train: 911, Val: 9, Test: 48


In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
import pandas as pd
from PIL import Image
import numpy as np
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report, confusion_matrix


In [3]:
import os
import pandas as pd

# Modify the create_dataset function to include "UNKNOWN" category
def create_dataset(folder_path):
    my_list = []
    for category in ['NORMAL', 'PNEUMONIA', 'UNKNOWN', 'TUBERCULOSIS']:
        category_path = os.path.join(folder_path, category)
        for file_name in os.listdir(category_path):
            file_path = os.path.join(category_path, file_name)
            # Ensure we're only adding image files
            if os.path.isfile(file_path) and file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                my_list.append([file_path, category])
    return pd.DataFrame(my_list, columns=['file_path', 'label'])

# Dataset paths
dataset_dir = r"D:\source\data\data"
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')
test_dir = os.path.join(dataset_dir, 'test')

# Create DataFrames for train, validation, and test datasets
train_df = create_dataset(train_dir)
val_df = create_dataset(val_dir)
test_df = create_dataset(test_dir)

# Convert labels to numeric: NORMAL -> 0, PNEUMONIA -> 1, UNKNOWN -> 2
train_df['label'] = train_df['label'].map({'NORMAL': 0, 'PNEUMONIA': 1, 'UNKNOWN': 2 , 'TUBERCULOSIS': 3})
val_df['label'] = val_df['label'].map({'NORMAL': 0, 'PNEUMONIA': 1, 'UNKNOWN': 2, 'TUBERCULOSIS': 3})
test_df['label'] = test_df['label'].map({'NORMAL': 0, 'PNEUMONIA': 1, 'UNKNOWN': 2, 'TUBERCULOSIS': 3})

# Print dataset sizes
print(f"Train set size: {len(train_df)}, Validation set size: {len(val_df)}, Test set size: {len(test_df)}")


Train set size: 13028, Validation set size: 761, Test set size: 1527


In [6]:
# Function to count categories in a given DataFrame
def count_categories(df, dataset_name):
    category_counts = df['label'].value_counts()
    print(f"{dataset_name} set:")
    print(f"  NORMAL: {category_counts.get(0, 0)}")  # NORMAL = 0
    print(f"  PNEUMONIA: {category_counts.get(1, 0)}")  # PNEUMONIA = 1
    print(f"  UNKNOWN: {category_counts.get(2, 0)}")  # UNKNOWN = 2 (added the UNKNOWN category)
    print(f"  TUBERCULOSIS: {category_counts.get(3, 0)}")  # TUBERCULOSIS = 3

# Count and display for train, validation, and test datasets
print("Image Counts per Category:")
count_categories(train_df, "Train")
count_categories(val_df, "Validation")
count_categories(test_df, "Test")




Image Counts per Category:
Train set:
  NORMAL: 4667
  PNEUMONIA: 3633
  UNKNOWN: 1155
  TUBERCULOSIS: 3573
Validation set:
  NORMAL: 274
  PNEUMONIA: 213
  UNKNOWN: 67
  TUBERCULOSIS: 207
Test set:
  NORMAL: 548
  PNEUMONIA: 427
  UNKNOWN: 135
  TUBERCULOSIS: 417


In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd

# Define Dataset class
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]  # Image path
        label = self.dataframe.iloc[idx, 1]     # Image label (NORMAL, PNEUMONIA, UNKNOWN)
        img = Image.open(img_path).convert('RGB')  # Convert the image to RGB if not already

        if self.transform:
            img = self.transform(img)

        return img, label



train_transform_resnet = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 for ResNet-18 input size
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB (3 channels)
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomRotation(5),  # Slight rotation between -5 and +5 degrees
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for ResNet
])


val_transform_resnet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB (3 channels)
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for ResNet
])

# Assuming train_df and val_df are already created with 'file_path' and 'label' columns
train_dataset_resnet = ImageDataset(train_df, transform=train_transform_resnet)
val_dataset_resnet = ImageDataset(val_df, transform=val_transform_resnet)

# DataLoader - For training and validation datasets
batch_size = 32

train_loader_resnet = DataLoader(train_dataset_resnet, batch_size=batch_size, shuffle=True)
val_loader_resnet = DataLoader(val_dataset_resnet, batch_size=batch_size, shuffle=False)

# Optionally print dataset sizes
print(f"Training dataset size: {len(train_loader_resnet)}")
print(f"Validation dataset size: {len(val_loader_resnet)}")



Training dataset size: 408
Validation dataset size: 24


In [9]:
# Check for multi-GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
# Function to prepare model for multi-GPU usage
def prepare_model_for_multigpu(model):
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        model = nn.DataParallel(model)
    model = model.to(device)
    return model

Device: cuda


In [11]:
import torch
import torch.nn as nn
import os
import time
from torch.utils.tensorboard import SummaryWriter

# Training function with TensorBoard logging and history tracking
def train_model_with_history_and_tensorboard(model, train_loader, val_loader, criterion, optimizer, num_epochs=30, device='cuda'):
    # Ensure model is moved to the correct device (GPU/CPU)
    model.to(device)

    # Initialize TensorBoard writer
    writer = SummaryWriter(log_dir='tensorboard_logs')

    # Track training and validation losses and accuracies
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    # Load model weights (if resuming from a pre-trained model)
    if os.path.exists('model/model_29.pt'):
        model.load_state_dict(torch.load('model/model_29.pt', map_location=device))
        print("Model weights loaded successfully.")
    else:
        print("No pre-trained weights found, starting from scratch.")
    
    model_dir = 'model'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    # Training loop for the specified number of epochs
    for epoch in range(num_epochs):
        start_epoch_time = time.time()  # Start time for the epoch

        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        # Training loop
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Track running loss and accuracy
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Calculate training loss and accuracy
        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        # Start time for validation phase
        start_val_time = time.time()

        # Validation loop
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        # Calculate validation loss and accuracy
        val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        # End time for validation phase
        end_val_time = time.time()
        validation_time = end_val_time - start_val_time

        # Calculate total time for the epoch
        end_epoch_time = time.time()
        epoch_time = end_epoch_time - start_epoch_time

        # Print results for the epoch
        print(f"Epoch [{epoch+1}/{num_epochs}] "
                f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
                f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}% | "
                f"Epoch Time: {epoch_time:.2f}s | Validation Time: {validation_time:.2f}s")

        # Log metrics to TensorBoard
        writer.add_scalar('Loss/Train', train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Train', train_accuracy, epoch)
        writer.add_scalar('Accuracy/Validation', val_accuracy, epoch)
        writer.add_scalar('Time/Epoch', epoch_time, epoch)
        writer.add_scalar('Time/Validation', validation_time, epoch)

        # Save the model after every epoch (ensure the directory exists)
        torch.save(model.state_dict(), f'{model_dir}/model_{epoch}.pt')

    # Close the TensorBoard writer
    writer.close()

    return train_losses, val_losses, train_accuracies, val_accuracies

# Example usage of the function
# Make sure your DataLoader objects (train_loader, val_loader) and model are defined
class_weights = [.70, 0.90, 1.0, 1.0]  
# Convert class weights to a tensor
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

# Define the weighted CrossEntropy loss function
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)


In [10]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Sample class distribution based on the dataset
class_labels = np.array([0, 1, 2 , 3])  # Example class labels
y_train = np.array([0]*4667 + [1]*3633 + [2]*1155 + [3]*3573)  # Example training set distribution

# Calculate class weights using sklearn
class_weights = compute_class_weight('balanced', classes=class_labels, y=y_train)

print(f"Class weights: {dict(zip(class_labels, class_weights))}")


Class weights: {np.int64(0): np.float64(0.6978787229483608), np.int64(1): np.float64(0.896504266446463), np.int64(2): np.float64(2.8199134199134197), np.int64(3): np.float64(0.9115589140778058)}


In [12]:
# Load pretrained ResNet-18 model
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(resnet18.fc.in_features, 4)  # Modify for binary classification
resnet18 = prepare_model_for_multigpu(resnet18)  # Apply multi-GPU


c:\Users\rifat\miniconda3\envs\mlweb\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\rifat\miniconda3\envs\mlweb\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Define optimizer for ResNet-18
optimizer_resnet = optim.Adam(resnet18.parameters(), lr=0.0001)

# Train the Pretrained ResNet-18 Model
print("Training Pretrained ResNet-18 Model...")
train_losses_resnet, val_losses_resnet, train_accuracies_resnet, val_accuracies_resnet = train_model_with_history_and_tensorboard(
    resnet18, train_loader_resnet, val_loader_resnet, criterion, optimizer_resnet, num_epochs=60
)


Training Pretrained ResNet-18 Model...
No pre-trained weights found, starting from scratch.
Epoch [1/60] Train Loss: 0.1452, Train Acc: 94.17% | Val Loss: 0.2122, Val Acc: 91.20% | Epoch Time: 244.38s | Validation Time: 12.80s
Epoch [2/60] Train Loss: 0.0818, Train Acc: 96.73% | Val Loss: 0.1319, Val Acc: 94.74% | Epoch Time: 241.64s | Validation Time: 12.85s
Epoch [3/60] Train Loss: 0.0676, Train Acc: 97.48% | Val Loss: 0.0852, Val Acc: 95.93% | Epoch Time: 247.88s | Validation Time: 13.22s
Epoch [4/60] Train Loss: 0.0622, Train Acc: 97.54% | Val Loss: 0.0722, Val Acc: 97.50% | Epoch Time: 249.32s | Validation Time: 12.91s
Epoch [5/60] Train Loss: 0.0501, Train Acc: 98.09% | Val Loss: 0.0614, Val Acc: 98.03% | Epoch Time: 242.23s | Validation Time: 12.58s
Epoch [6/60] Train Loss: 0.0447, Train Acc: 98.17% | Val Loss: 0.3728, Val Acc: 92.64% | Epoch Time: 242.93s | Validation Time: 12.73s
Epoch [7/60] Train Loss: 0.0411, Train Acc: 98.41% | Val Loss: 0.0592, Val Acc: 98.03% | Epoch Tim